# Recommending Movies
In this chapter you will be introduced to the MovieLens dataset. You will walk through how to assess it's use for ALS, build out a full cross-validated ALS model on it, and learn how to evaluate it's performance. This will be the foundation for all subsequent ALS models you build using Pyspark.

## Viewing the MovieLens Data
Familiarize yourself with the ratings dataset provided here. Would you consider the data to be implicit or explicit ratings?

In [2]:
# File Path
file_path = ".../data/datacamp/"

# Load in data
ratings = spark.read.parquet(file_path + 'ratings')

# Look at the column names
print(ratings.columns)

# Look at the first few rows of data
ratings.show(5)

['userId', 'movieId', 'rating', 'timestamp']
+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|   514|    799|   3.0| 853893138|
|   560|   2513|   4.0|1452851841|
|   253|   2688|   1.5|1215832619|
|    15|  34334|   1.5|1367765144|
|   285|   1748|   2.0| 965088503|
+------+-------+------+----------+
only showing top 5 rows



Nice. This dataset includes ratings from the customers. This indicates that these are explicit ratings.

## Calculate sparsity
As you know, ALS works well with sparse datasets. Let's see how much of the ratings matrix is actually empty.

Remember that sparsity is calculated by the number of cells in a matrix that contain a rating divided by the total number of values that matrix could hold given the number of users and items (movies). In other words, dividing the number of ratings present in the matrix by the product of users and movies in the matrix and subtracting that from 1 will give us the sparsity or the percentage of the ratings matrix that is empty.

In [3]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.36% empty.


That's right, this matrix is more than 98% empty. That's a lot of missing data.

## The GroupBy and Filter Methods
Now that we know a little more about the dataset, let's look at some general summary metrics of the ratings dataset and see how many ratings the movies have and how many ratings each users has provided.

Two common methods that will be helpful to you as you aggregate summary statistics in Spark are the .filter() and the .groupBy() methods. The .filter() method allows you to filter out any data that doesn't meet your specified criteria.

In [4]:
# Import the requisite packages
from pyspark.sql.functions import col

# View the ratings dataset
ratings.show(5)

# Filter to show only userIds less than 100
ratings.filter(col("userId") < 100).show(5)

# Group data by userId, count song plays
ratings.groupBy("userId").count().show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|   514|    799|   3.0| 853893138|
|   560|   2513|   4.0|1452851841|
|   253|   2688|   1.5|1215832619|
|    15|  34334|   1.5|1367765144|
|   285|   1748|   2.0| 965088503|
+------+-------+------+----------+
only showing top 5 rows

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|    15|  34334|   1.5|1367765144|
|    30|    532|   2.0| 945277742|
|    74|   1722|   5.0| 942704332|
|     8|   5952|   4.0|1154464762|
|    11|  26614|   5.0|1391658574|
+------+-------+------+----------+
only showing top 5 rows

+------+-----+
|userId|count|
+------+-----+
|   148|  132|
|   540|   20|
|   597|  202|
|   406|   73|
|   442|  225|
+------+-----+
only showing top 5 rows



## MovieLens Summary Statistics
Let's take the groupBy() method a bit further.

Once you've applied the .groupBy() method to a dataframe, you can subsequently run aggregate functions such as .sum(), .avg(), .min() and have the results grouped. This exercise will walk you through how this is done. The min and avg functions have been imported from pyspark.sql.functions for you.

In [5]:
from pyspark.sql.functions import min, avg
# Min num ratings for movies
print("Movie with the fewest ratings: ")
ratings.groupBy("movieId").count().select(min("count")).show()

# Avg num ratings per movie
print("Avg num ratings per movie: ")
ratings.groupBy("movieId").count().select(avg("count")).show()

# Min num ratings for user
print("User with the fewest ratings: ")
ratings.groupBy("userId").count().select(min("count")).show()

# Avg num ratings per users
print("Avg num ratings per user: ")
ratings.groupBy("userId").count().select(avg("count")).show()

Movie with the fewest ratings: 
+----------+
|min(count)|
+----------+
|         1|
+----------+

Avg num ratings per movie: 
+------------------+
|        avg(count)|
+------------------+
|11.030664019413193|
+------------------+

User with the fewest ratings: 
+----------+
|min(count)|
+----------+
|        20|
+----------+

Avg num ratings per user: 
+------------------+
|        avg(count)|
+------------------+
|149.03725782414307|
+------------------+



That's right. Users have at least 20 ratings and on average of 149 ratings. And movies have at least 1 rating with an average of 11 ratings.

## View Schema
As you know from previous chapters, Spark's implementation of ALS requires that movieIds and userIds be provided as integer datatypes. Many datasets need to be prepared accordingly in order for them to function properly with Spark. A common issue is that Spark thinks numbers are strings, and vice versa.

Here, you'll use the .cast() method to address these types of problems. Let's take a look at the schema of the dataset to ensure it's in the correct format.

In [6]:
# Use .printSchema() to see the datatypes of the ratings dataset
ratings.printSchema()

# Tell Spark to convert the columns to the proper data types
ratings = ratings.select(ratings.userId.cast("integer"), ratings.movieId.cast("integer"), ratings.rating.cast("double"))

# Call .printSchema() again to confirm the columns are now in the correct format
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



## Create test/train splits and build your ALS model
You already know how to build an ALS model, having done it in the previous chapter. We will do that again, but we'll take some additional steps to fully build out a cross-validated model.

First, let's import the requisite functions and create our train and test data sets in preparation for the cross validation step.

In [7]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False)

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

## Tell Spark how to tune your ALS model
Now we'll need to create a ParamGrid to tell Spark what hyperparameters we want it to tune, how to tune them, and then build out an evaluator so Spark can know how to measure the algorithm's performance.

In [8]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
           .addGrid(als.rank, [10, 50, 75, 100]) \
           .addGrid(als.maxIter, [5, 50, 100, 200]) \
           .addGrid(als.regParam, [.01, .05, .1, .15]) \
           .build()

# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  64


## Build your cross validation pipeline
Now that we have our data, our train/test splits, our model, and our hyperparameter values, let's tell Spark how to cross validate our model so it can find the best combination of hyperparameters and return it to us.

In [9]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# Confirm cv was built
print(cv)

CrossValidator_32ba8da08f78


## Best Model and Best Model Parameters
Now that we have our cross validator, cv, built out, we can tell Spark to take our data, fit the ALS algorithm to it, and try the different combinations of hyperparameter values from our param_grid so that it can identify what values provide the smallest RMSE. Unfortunately, this takes too long to complete here, but for your reference, this is how it is done:

```python
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel
```
This code has been run separately, and the best_model has been identified and saved for you to use. Use the commands given to extract the parameters of the model.

In [ ]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# Print "Rank"
print("  Rank:", best_model.getRank())

# Print "MaxIter"
print("  MaxIter:", best_model.getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model.getRegParam())

Excellent work! If you'll notice, the best hyperparameter values were all in the middle of the ranges we provided. If they were on the low or high end, we would simply adjust our ranges accordingly. Given that they were in the middle, we could tune even further by narrowing our range to get even more precise hyperparameter values.

## Generate predictions and calculate RMSE
Now that we have a model that is trained on our data and tuned through cross validation, we can see how it performs on the test dataframe. To do this, we'll calculate the RMSE.

As a side note, the generation of test predictions takes more than a few minutes with this dataset. For this reason, the test predictions have been generated already and are provided here as a dataframe called test_predictions. For your reference, they are generated using this code: 

```python
test_predictions = best_model.transform(test).
```

In [ ]:
# View the predictions 
test_predictions.show(5)

# Calculate the RMSE of test_predictions
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

Excellent. Remember that the RMSE is a rather subjective metric. Would you say that the RMSE in this case is sufficient to make meaningful recommendations? 

Interpretation: An RMSE of 0.633 means that on average the model predicts 0.633 above or below values of the original ratings matrix.

## Do Recommendations Make Sense
Now that we have an understanding of how well our model performed, and have some confidence that it will provide recommendations that are relevant to users, let's actually look at recommendations made to a user and see if they make sense.

The original ratings data is provided here as original_ratings. Take a look at user 60 and user 63's original ratings, and compare them to what ALS recommended for them. In your opinion, are the recommendations consistent with their original preferences?

In [ ]:
# Look at user 60's ratings
print("User 60's Ratings:")
original_ratings.filter(col("userId") == 60).sort("rating", ascending = False).show()

# Look at the movies recommended to user 60
print("User 60s Recommendations:")
recommendations.filter(col("userId") == 60).show(5)

# Look at user 63's ratings
print("User 63's Ratings:")
original_ratings.filter(col("userId") == 63).sort("rating", ascending = False).show(5)

# Look at the movies recommended to user 63
print("User 63's Recommendations:")
recommendations.filter(col("userId") == 63).show(5)

Great work. Does it look like the model picked up on user 60's preference for drama, crime, and comedy or user 63's preference for action, adventure, and drama?